<a href="https://colab.research.google.com/github/marcomedugno/marcomedugno/blob/main/Spark_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spark SQL on Colab


##Marco Medugno



In [ ]:
# install Java Virtual Machine (JVM) from OpenJDK
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
# download and decompress Apache Spark with Hadoop from https://spark.apache.org/downloads.html
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz

In [ ]:
# set environment path
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'
os.environ['SPARK_HOME'] = '/content/spark-3.2.1-bin-hadoop3.2'

In [ ]:
# install and import findspark to locate Spark on the system
!pip install -q findspark
import findspark
findspark.init()
findspark.find()

'/content/spark-3.2.1-bin-hadoop3.2'

In [ ]:
# Spark resilient distributed dataset (RDD)
import pyspark
sc = pyspark.SparkContext(appName='Act7')
data = list(range(7))
rdd = sc.parallelize(data)
rdd.getNumPartitions(), rdd.collect()

(2, [0, 1, 2, 3, 4, 5, 6])

In [ ]:
# import SparkSession from pyspark.sql and create a SparkSession, which is the entry point to Spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local').appName('Colab').config('spark.ui.port', '4050').getOrCreate()
spark

In [ ]:
# authorize Colab to access Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# load data into PySpark
customer = spark.read.json('/content/drive/My Drive/hw4_customer.json')
order = spark.read.json('/content/drive/My Drive/hw4_order.json')
product = spark.read.json('/content/drive/My Drive/hw4_product.json')
orderline = spark.read.json('/content/drive/My Drive/hw4_orderline.json')


**1. What are the description, finish and standard price of product, which standard price is less than $275 in the ascending order?**

In [ ]:
product.filter('productStandardPrice < 275').select('productDescription', 'productFinish', 'productStandardPrice').orderBy(product.productStandardPrice.asc()).show()

+-------------------+-------------+--------------------+
| productDescription|productFinish|productStandardPrice|
+-------------------+-------------+--------------------+
|        48 Bookcase|       Walnut|               150.0|
|         Nightstand|       Cherry|               150.0|
|   Cherry End Table|       Cherry|               175.0|
|        48 Bookcase|          Oak|               175.0|
|Birch Coffee Tables|        Birch|               200.0|
|        96 Bookcase|          Oak|               200.0|
|        96 Bookcase|       Walnut|               225.0|
|     Pine End Table|         Pine|               256.0|
+-------------------+-------------+--------------------+



**2. What are the description, finish and standard price of all desks and all tables that cost more than $300 in the descending order?**

In [ ]:
from pyspark.sql.functions import col
product.filter(col("productDescription").rlike("Desk|Table") & ~col("productStandardPrice").between(0,300)).select('productDescription', 'productFinish', 'productStandardPrice').orderBy(product.productStandardPrice.desc()).show()

+------------------+-------------+--------------------+
|productDescription|productFinish|productStandardPrice|
+------------------+-------------+--------------------+
| Oak Computer Desk|          Oak|               750.0|
|     Writer's Desk|          Oak|               325.0|
+------------------+-------------+--------------------+



**3. What are the description and finish of product that has been ordered in the ascending order of finish then description?**

In [ ]:
product.join(orderline,['productID'],'leftsemi').select('productDescription', 'productFinish').orderBy(col("productFinish"),col("productDescription").asc()).show()

+--------------------+-------------+
|  productDescription|productFinish|
+--------------------+-------------+
|    8-Drawer Dresser|        Birch|
| Birch Coffee Tables|        Birch|
|    Cherry End Table|       Cherry|
|Entertainment Center|       Cherry|
|         48 Bookcase|          Oak|
|   Oak Computer Desk|          Oak|
|       Writer's Desk|          Oak|
|         48 Bookcase|       Walnut|
+--------------------+-------------+



**4. What are the name (no truncation), city and state of customer in Florida, Texas, California or Hawaii in the ascending order of name?**

In [ ]:
# display rows from top
customer.filter(col("customerState").isin("FL","TX","CA","HI")).select('customerName','customerCity','customerState').orderBy(customer.customerName.asc()).show(truncate=False)

+------------------------+------------+-------------+
|customerName            |customerCity|customerState|
+------------------------+------------+-------------+
|California Classics     |Santa Clara |CA           |
|Contemporary Casuals    |Gainesville |FL           |
|Impressions             |Sacramento  |CA           |
|Kaneohe Homes           |Kaneohe     |HI           |
|M and H Casual Furniture|Clearwater  |FL           |
|Seminole Interiors      |Seminole    |FL           |
|Value Furniture         |Plano       |TX           |
+------------------------+------------+-------------+



**5. How many customers in each of the state Florida, Texas, California or Hawaii?**

In [ ]:
customer.filter(col("customerState").isin("FL","TX","CA","HI")).groupBy('customerState').count().show()

+-------------+-----+
|customerState|count|
+-------------+-----+
|           CA|    2|
|           TX|    1|
|           FL|    3|
|           HI|    1|
+-------------+-----+



**6. What is the average standard price for all products in inventory?**

In [ ]:
from pyspark.sql.functions import avg
averageValue = product.agg(avg('productStandardPrice')).collect()[0][0]
print(f'average price = {averageValue}')

average price = 534.6315789473684


**7. What are the product description, product finish, and the price higher than the average standard price for all products in inventory, in the descending order of price difference?**

In [ ]:
product.withColumn("priceDifference", product.productStandardPrice-averageValue).select('productDescription','productFinish','priceDifference').orderBy(product.productStandardPrice.desc()).show()

+--------------------+-------------+-------------------+
|  productDescription|productFinish|    priceDifference|
+--------------------+-------------+-------------------+
|Entertainment Center|       Cherry| 1115.3684210526317|
|              Amoire|       Walnut|  665.3684210526316|
|7' Grandfather Clock|          Oak|  565.3684210526316|
|6' Grandfather Clock|          Oak| 355.36842105263156|
|    8-Drawer Dresser|          Oak| 265.36842105263156|
|    8-Drawer Dresser|        Birch| 215.36842105263156|
|   Oak Computer Desk|          Oak| 215.36842105263156|
|    4-Drawer Dresser|          Oak| -34.63157894736844|
|High Back Leather...|      Leather|-172.63157894736844|
|       Writer's Desk|          Oak|-209.63157894736844|
|       Writer's Desk|        Birch|-234.63157894736844|
|      Pine End Table|         Pine|-278.63157894736844|
|         96 Bookcase|       Walnut|-309.63157894736844|
|         96 Bookcase|          Oak|-334.63157894736844|
| Birch Coffee Tables|        B

**8. What are the order id, order date, the customer name (no truncation), and the overall total price for each order, in the ascending order of order id?**

In [ ]:
from pyspark.sql.functions import sum
order2=order.join(customer,['customerID'],'inner').select('orderID', 'orderDate','customerName')
orderline2=orderline.join(product,['productID'],'inner').select('orderID','orderedQuantity','productStandardPrice')
orderline2=orderline2.withColumn("cost", orderline2.orderedQuantity*orderline2.productStandardPrice)
orderline2=orderline2.groupBy('orderID').agg(sum('cost'))
order2.join(orderline2,['orderID'],'inner').withColumnRenamed('sum(cost)','totalPrice').orderBy(order2.orderID.asc()).show(truncate=False)

+-------+----------+------------------------+----------+
|orderID|orderDate |customerName            |totalPrice|
+-------+----------+------------------------+----------+
|1001   |2010-10-21|Contemporary Casuals    |2400.0    |
|1002   |2010-10-21|California Classics     |3750.0    |
|1003   |2010-10-22|Mountain Scenes         |2250.0    |
|1004   |2010-10-22|Impressions             |1850.0    |
|1005   |2010-10-24|Home Furnishings        |4950.0    |
|1006   |2010-10-24|Value Furniture         |2600.0    |
|1007   |2010-10-27|American Euro Lifestyles|925.0     |
|1008   |2010-10-30|Battle Creek Furniture  |2775.0    |
|1009   |2010-11-05|Eastern Furniture       |3750.0    |
|1010   |2010-11-05|Contemporary Casuals    |1750.0    |
+-------+----------+------------------------+----------+



**9. What are the id, name (no truncation), full address, and number of orders (0 if no order) for all customers, in the ascending order of customer id?**

In [ ]:
from pyspark.sql.functions import concat_ws,col
customerCount=order.groupby('customerID').count()
customer.join(customerCount,['customerID'],'full').na.fill(value=0).select('customerID','customerName',concat_ws(", ",customer.customerAddress,customer.customerCity,customer.customerState).alias('Address'),'count').show(truncate=False)

+----------+------------------------+---------------------------------------+-----+
|customerID|customerName            |Address                                |count|
+----------+------------------------+---------------------------------------+-----+
|1         |Contemporary Casuals    |1355 S Hines Blvd, Gainesville, FL     |2    |
|2         |Value Furniture         |15145 S.W. 17th St., Plano, TX         |1    |
|3         |Home Furnishings        |1900 Allard Ave., Albany, NY           |1    |
|4         |Eastern Furniture       |1925 Beltline Rd., Carteret, NJ        |1    |
|5         |Impressions             |5585 Westcott Ct., Sacramento, CA      |1    |
|6         |Furniture Gallery       |325 Flatiron Dr., Boulder, CO          |0    |
|7         |Period Furniture        |394 Rainbow Dr., Seattle, WA           |0    |
|8         |California Classics     |816 Peach Rd., Santa Clara, CA         |1    |
|9         |M and H Casual Furniture|3709 First Street, Clearwater, FL      